<a href="https://colab.research.google.com/github/AvtnshM/SSL/blob/main/Self_Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [2]:

# Used ONLY to load dataset
base_transform = transforms.ToTensor()

# Used to create SSL views
ssl_transform = transforms.Compose([
    transforms.RandomResizedCrop(32),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
    transforms.ToTensor()
])

to_pil = transforms.ToPILImage()

In [3]:

dataset = datasets.CIFAR10(
    root="./data",
    train=True,
    download=True,
    transform=base_transform   # ← important
)

In [4]:

loader = DataLoader(
    dataset,
    batch_size=256,
    shuffle=True,
    num_workers=2,
    drop_last=True
)

print("DataLoader created. Number of batches:", len(loader))

DataLoader created. Number of batches: 195


In [5]:

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 64, 3, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 128, 3, stride=2),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)
        )

    def forward(self, x):
        x = self.net(x)
        return x.view(x.size(0), -1)

In [6]:

class Predictor(nn.Module):
    def __init__(self, dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, dim),
            nn.ReLU(),
            nn.Linear(dim, dim)
        )

    def forward(self, x):
        return self.net(x)

In [7]:

def ssl_loss(p, z):
    p = F.normalize(p, dim=1)
    z = F.normalize(z, dim=1)
    return F.mse_loss(p, z)

In [8]:

# Online encoder (learns with gradients)
encoder = Encoder().to(device)

# Predictor head (learns with gradients)
predictor = Predictor(dim=128).to(device)

# Target encoder (EMA, NO gradients)
target_encoder = Encoder().to(device)

# Initialize target encoder = online encoder
target_encoder.load_state_dict(encoder.state_dict())

# Disable gradients for target encoder
for param in target_encoder.parameters():
    param.requires_grad = False

# Optimizer (ONLY online encoder + predictor)
optimizer = torch.optim.Adam(
    list(encoder.parameters()) + list(predictor.parameters()),
    lr=1e-3
)

# EMA decay rate (standard value)
ema_tau = 0.996

In [9]:

@torch.no_grad()
def update_target_encoder(online_encoder, target_encoder, tau):
    for online_param, target_param in zip(
        online_encoder.parameters(),
        target_encoder.parameters()
    ):
        target_param.data = (
            tau * target_param.data +
            (1.0 - tau) * online_param.data
        )

In [10]:

epochs = 10

for epoch in range(epochs):
    total_loss = 0.0

    for images, _ in loader:
        images = images.to(device)

        # TRUE two stochastic views
        view1 = torch.stack([
            ssl_transform(to_pil(img.cpu())) for img in images
        ]).to(device)

        view2 = torch.stack([
            ssl_transform(to_pil(img.cpu())) for img in images
        ]).to(device)

        # Online encoder
        z1 = encoder(view1)
        z2 = encoder(view2)

        # Predictor (asymmetry)
        p1 = predictor(z1)
        p2 = predictor(z2)

        # EMA target encoder (NO gradients)
        with torch.no_grad():
            t1 = target_encoder(view1)
            t2 = target_encoder(view2)

        # BYOL loss
        loss = ssl_loss(p1, t2.detach()) + ssl_loss(p2, t1.detach())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # EMA update (CRITICAL)
        update_target_encoder(encoder, target_encoder, ema_tau)

        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    print(f"Epoch [{epoch+1}/{epochs}] Loss: {avg_loss:.4f}")

Epoch [1/10] Loss: 0.0010
Epoch [2/10] Loss: 0.0000
Epoch [3/10] Loss: 0.0000
Epoch [4/10] Loss: 0.0000
Epoch [5/10] Loss: 0.0000
Epoch [6/10] Loss: 0.0000
Epoch [7/10] Loss: 0.0000
Epoch [8/10] Loss: 0.0000
Epoch [9/10] Loss: 0.0001
Epoch [10/10] Loss: 0.0001
